# Computerphysik Programmiertutorial 8b
Prof. Dr. Matteo Rizzi und Dr. Markus Schmitt - Institut für Theoretische Physik, Universität zu Köln
&nbsp;

**Github**: [https://github.com/markusschmitt/compphys2022](https://github.com/markusschmitt/compphys2022)

**Inhalt dieses Notebooks**: Virtuelles Pohl'sches Rad: Einbinden von Code aus externen Dateien, Diskrete Fouriertransformation, Fehlerfortpflanzung, least-squares Fits

# Ein virtuelles Experiment

<img src="https://lp.uni-goettingen.de/get/bigimage/181"></img>

In [ ]:
using Plots, LaTeXStrings

include("pohl_experiment.jl")

using .PohlExperiment

## Ein virtuelles Experiment ausführen

Daten inspizieren:

In [ ]:
plot(data.t,data.phi,marker=:+)

xlabel!(L"t")
ylabel!(L"$\varphi(t)$")

## Eigenfrequenz bestimmen

Fouriertransformation des Schwingungssignals:

Die diskrete Fouriertransformation eines Signals $a=(a_1,\ldots,a_N)$ ergibt die Fourierkoeffizienten

$$
\hat a_k=\sum_{j=1}^{N}e^{-2\pi i\frac{jk}{N}}a_j
$$

Der Index $k$ ist also der Frequenz $\omega_k=\frac{2\pi k}{N}$ zugeordnet. Für ein reelles Signal gilt $\hat a_k=(\hat a_{N-k})^*$.

Diskrete Fouriertransformation ist im Julia Paket `FFTW` implementiert ([siehe hier](https://juliamath.github.io/FFTW.jl/stable/)). Fouriertransformation eines reellen Signals wird mit der Funktion `rfft` ausgeführt:

Spektrum plotten:

In [ ]:
# Frequenzen berechnen
dt = data.t[2]-data.t[1]
k_values = collect(1:length(phi_omega)).-1
omega_k = k_values*2*pi/length(data.phi)/dt

plot(omega_k, abs.(phi_omega), marker=:o)

xlabel!(L"Frequenz $\omega_k$")
ylabel!(L"Fourierkoeffizient $\hat a(\omega_k)$");
xlims!(0,3)

Bestimme den Parameter $\Omega_0$ als den Wert, bei dem $A(\omega)$ maximal ist.

Dieses Ergebnis ist fehlerbehaftet. Das können wir in Julia berücksichtigen und das Paket `Measurements` verwenden ([siehe hier](https://juliaphysics.github.io/Measurements.jl/stable/)). `Measurements` führt einen Datentyp für fehlerbehaftete Zahlen ein und ermöglicht damit sehr einfach die Fehlerfortpflanzung zu berechnen.

Da wir unser Ergebnis für $\Omega_0$ durch Ablesen des Maximums erhalten haben, ist es sinnvoll als Fehler die Auflösung auf der Frequenz-Achse anzugeben. Dafür nutzen wir `measurement` aus dem `Measurements` Paket:

**Beispiel zur Fehlerfortpflanzung:**

Typischerweise kann der Hersteller des Pohl'schen Rades das zugehörige Trägheitsmoment $\Theta$ angeben. In unserem Modul `PohlExperiment` haben wir eine solche Angabe mit eingebaut, inklusive einer Fehlerangabe:

Mit dieser Angabe und unserem Ergebnis für $\Omega_0$ können wir die "Federkonstante" $k$ des Rades ausrechnen:

$$
    k = \Omega_0^2\Theta
$$

Wenn wir mit `measurement`s rechnen, wird die zugehörige Fehlerfortpflanzung automatisch mitberechnet:

## Analyse inklusive Dämpfung und Antrieb

Die maximale Schwingungsamplitude des Pohl'sche Rades mit Dämpfung und Antrieb ist im Anschluss an die Einschwingphase gegeben durch

$$
\varphi_{max}(\Omega_F)=\frac{F}{\sqrt{(\Omega_F^2-\Omega_0^2)^2+\gamma^2\Omega_0^2}}
$$

Wir werden jetzt eine Reihe virtueller Experimente mit verschiedenen $\Omega_F$ durchführen um $\varphi_{max}(\Omega_F)$ zu messen. Anschließend werden wir eine Funktion an die Daten fitten um die Parameter $\gamma$ und $F$ zu bestimmen.

Wir beginnen also mit der Datenaufnahme:

Daten visualisieren:

In [ ]:
plot()
for d in dataList
    plot!(d.t, d.phi, label="")
end

xlabel!(L"time $t$")
ylabel!(L"Amplitude $\varphi(t)$")

Nun wollen wir für jede Antriebsfrequenz die konstante Schwingungsamplitude nach der Einschwingphase bestimmen. Dafür definieren wir zunächst eine Funktion, die aus einem Array von Werten die lokalen Maxima heraussucht:

In [ ]:
function get_absolute_extrema(a)

    a_abs = abs.(a) # Wir wollen den Betrag betrachten
    
    # In dieser Liste werden wir die Extremwerte sammeln
    extrema = []
    
    # Schleife über Array-Einträge
    for i in 2:length(a_abs)-1
        
        # Testen ob der aktuelle eintrag ein Maximum ist
        if a_abs[i]-a_abs[i-1] > 0 && a_abs[i+1] - a_abs[i] < 0
            # Maxima der Liste hinzufügen
            push!(extrema, a_abs[i])
        end
    end
    
    return extrema
    
end

Nun können wir die Extrema unserer "gemessenen" Daten suchen und so ein Ergebnis für $\varphi(\Omega_F)$ bekommen:

In [ ]:
using Statistics

amplitude = []
amplitude_error = []

I0 = Int(50 / dt) # Start Index: ignoriere Einschwingzeit

for d in dataList
    
    A = get_absolute_extrema(d.phi[I0:end])
    
    push!(amplitude, mean(A))                        # Mittelwert der Amplitude
    push!(amplitude_error, std(A)/sqrt(length(A)))   # Standardfehler des Mittelwerts

end

# Plot
plot(Omega_F_list, amplitude, yerror=amplitude_error, marker=:o, markersize=1)
xlabel!(L"Antriebsfrequenz $\Omega_F$")
ylabel!(L"Amplitude $\varphi(\Omega_F)$")

Schließlich wollen wir noch einen Fit an die Datenpunkte machen, um die verbleibenden Parameter zu extrahieren. Dafür verwenden wir das Paket `LsqFit` ([siehe hier](https://julianlsolvers.github.io/LsqFit.jl/latest/)).

Damit können wir leicht "least squares" Fits durchführen. Zunächst müssen wir die Modellfunktion (den Ansatz) definieren:

In [ ]:
Omega_0_val = Omega_0.val # Wert unseres Ergebnisses für Omega_0

function model(omega, p)
    
    global Omega_0_val
    
    gamma = p[1]
    F0 = p[2]
    
    return @. F0 / sqrt((omega^2 - Omega_0_val^2)^2 + gamma^2 * omega^2)
end

Dann können wir den Fit durchführen:

Ergebnis: Die Variable `fit` hat nun ein Feld `param`, das das Ergebnis für die gefitteten Parameter enthält. Außerdem gibt die Funktion `estimate_covar` die Kovarianzmatrix aus, woraus sich die Fehlerabschätzung ergibt. Damit können wir das Ergebnis wieder als `measurement`s behandeln:

In [ ]:
function get_error(fit, i)
    return sqrt(estimate_covar(fit)[i,i])
end

errors = [get_error(fit, i) for i in 1:2]

gamma_fit = measurement(fit.param[1], errors[1])
F_fit = measurement(fit.param[2], errors[2])

println("gamma = ", gamma_fit)
println("F = ", F_fit)

Plot:

In [ ]:
plot(Omega_F_list, amplitude, yerror=amplitude_error, marker=".", markersize=1)
plot!(collect(0:0.01:pi), model(collect(0:0.01:pi), [gamma_fit.val, F_fit.val]))
xlabel!(L"Antriebsfrequenz $\Omega_F$")
ylabel!(L"Amplitude $\varphi(\Omega_F)$")

## Ergebnisse

In [ ]:
println("Exact:")
println("Omega_0 = ", PohlExperiment.Omega_0)
println("gamma = ", PohlExperiment.gamma[2])
println("F = ", PohlExperiment.F)
println("")
println("Unsere Auswertung:")
println("Omega_0 = ", Omega_0)
println("gamma = ", gamma_fit)
println("F = ", F_fit)